In [1]:
import pickle
df = pickle.load(open('Clayton_SG_df_drop.pkl', 'br'))
df['long_post'] = df['long_post'].apply(', '.join)
df['long_post'] = df['long_post'].str.replace(',', '')
df.sample(5)

,long_post,post_number,group
21961,struggling anxiety years gets worse get older ...,60926,Anxiety
10345,ive sodas morning theres jugs tea work im hopi...,377042,Caffeine
26365,today well yesterday wed feb th pm est somewha...,68427,Video Game Addiction
28887,realise something im going get something thats...,547768,Widow
24333,looking anyone camptocormia knows someone deal...,532870,Parkinsons


In [2]:
from sklearn.model_selection import train_test_split
X = df.long_post
y = df.group
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(19543,)
(4886,)
(19543,)
(4886,)


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(19543, 36145)

In [5]:
# TF-IDF: term-frequency times inverse document frequency
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(19543, 36145)

In [6]:
# Classifier using Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [7]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])

text_clf = text_clf.fit(X_train, y_train)

In [8]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.36962750716332377

In [9]:
# support vector machine classifier
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, max_iter=5, tol=None, random_state=42)),
])

_ = text_clf_svm.fit(X_train, y_train)

predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.6377404830126893

In [10]:
# Tuning hyperparameters
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
}

In [11]:
# Grid search with Naive Bayes
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [12]:
%%time
gs_clf = gs_clf.fit(X_train, y_train)
print(gs_clf.best_score_)
gs_clf.best_params_

0.47464565317504986
CPU times: user 2.67 s, sys: 308 ms, total: 2.98 s
Wall time: 22.5 s


In [13]:
# Grid search with SVM classifier
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf-svm__alpha': (1e-2, 1e-3),
}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

In [14]:
%%time
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
gs_clf_svm.best_params_

0.5988333418615361
CPU times: user 1min 25s, sys: 8.96 s, total: 1min 34s
Wall time: 2min 4s


In [15]:
import nltk
nltk.download('stopwords')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('mnb', MultinomialNB(fit_prior=False)),
])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
%%time
text_mnb_stemmed = text_mnb_stemmed.fit(X_train, y_train)

CPU times: user 11.4 s, sys: 146 ms, total: 11.5 s
Wall time: 11.5 s


In [17]:
predicted_mnb_stemmed = text_mnb_stemmed.predict(X_test)
np.mean(predicted_mnb_stemmed == y_test)

0.4950880065493246

In [18]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [19]:
vectorizers = [
    TfidfVectorizer(stop_words='english',
                    max_features=None),
    CountVectorizer(stop_words='english',
                   max_features=None)
]

classifiers = [
    MultinomialNB(),
    LinearSVC(),
    LogisticRegression(),
    RandomForestClassifier()
]

clf_names = [
         "Naive Bayes",
         "Linear SVC",
         "Logistic Regression",
         "Random Forest",
        ]

vect_names = [
    "TfidfVectorizer",
    "CountVectorizer"
]

clf_params = [
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__alpha': (1e-2, 1e-3)},
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__C': (np.logspace(-5, 1, 5))},
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__C': (np.logspace(-5, 1, 5))},
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__max_depth': (1, 2)},
             ]

In [20]:
%%time
models = []
for classifier, clf_name, params in zip(classifiers, 
                                        clf_names, 
                                        clf_params):
    for vectorizer, vect_name in zip(vectorizers, 
                                     vect_names):
        pipe = Pipeline([
            ('vect', vectorizer),
            ('clf', classifier),
        ])
        gs = GridSearchCV(pipe, 
                          param_grid=params, 
                          n_jobs=-1,
                          scoring='accuracy',
                          cv=5,
                          verbose=10)
        
        gs.fit(df.long_post, df.group)
        score = gs.best_score_
        print(f'''
Classifier: {clf_name}
Vectorizer: {vect_name}
Score: {gs.best_score_:.4f}
Params: {gs.best_params_}
------------------------------
            ''')
        models.append((clf_name, vect_name, gs.best_score_, gs.best_params_))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.001, vect__ngram_range=(1, 1) ......................
[CV] clf__alpha=0.001, vect__ngram_range=(1, 1) ......................
[CV] clf__alpha=0.001, vect__ngram_range=(1, 1) ......................
[CV] clf__alpha=0

[Parallel(n_jobs=-1)]: Done   4 out of  20 | elapsed:    5.2s remaining:   20.6s


[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.5169031719532554, total=   3.3s
[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.7862595419847328, total=   3.4s
[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.52768201176232, total=   3.3s
[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.5090871540685667, total=   3.3s
[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.5016373311502251, total=   3.4s


[Parallel(n_jobs=-1)]: Done   7 out of  20 | elapsed:    5.8s remaining:   10.8s
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed:    5.9s remaining:    5.9s


[CV]  clf__alpha=0.01, vect__ngram_range=(1, 2), score=0.5481646724802272, total=  13.9s
[CV]  clf__alpha=0.01, vect__ngram_range=(1, 2), score=0.5346410684474123, total=  13.9s
[CV]  clf__alpha=0.01, vect__ngram_range=(1, 2), score=0.5247646336471551, total=  14.0s


...

[CV]  clf__max_depth=1, vect__ngram_range=(1, 2), score=0.01669449081803005, total=   6.4s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.02249899558055444, total=   6.8s


[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:   10.1s remaining:    2.5s


[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.030825390387345365, total=   4.4s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.03151862464183381, total=   4.4s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.028500619578686492, total=   5.4s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.02003338898163606, total=   5.4s


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   11.9s finished



Classifier: Random Forest
Vectorizer: CountVectorizer
Score: 0.0451
Params: {'clf__max_depth': 2, 'vect__ngram_range': (1, 1)}
------------------------------
            
CPU times: user 59min 8s, sys: 3min 8s, total: 1h 2min 16s
Wall time: 2h 55min 2s


In [21]:
models = sorted(models, key=lambda tup: tup[2])
print('And the winner is...')
print()
print('Classifier:', models[-1][0])
print('Vectorizer:', models[-1][1])
print('Score:', models[-1][2])
print('Params:', models[-1][3])

And the winner is...

Classifier: Linear SVC
Vectorizer: TfidfVectorizer
Score: 0.6904908101027467
Params: {'clf__C': 10.0, 'vect__ngram_range': (1, 2)}
